In [2]:
from bs4 import BeautifulSoup
import mechanicalsoup
import requests
import time, os
from selenium import webdriver
import math
import pickle
import re
import numpy as np
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as pd_sql

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.tummee.com/login?")

cookies = pickle.load(open("cookies.pickle", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get("https://www.tummee.com/sequences/teachers/daily")

In [4]:
f = open("restorative_urls", "rb")
restorative_urls = pickle.load(f)
f.close()

In [5]:
# note, i'm defining this in each ipynb instead of as a .py module
# because it allows it to all run inside a single chrome window instead
# of opening a new window each time (like it would do if calling .py function every time) 
# which is much faster

def get_poses(url): 
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try: 
        title = re.sub(r"[\t\n0-9.]", "", soup.find("h1").text.strip())
        find_poses = soup.find_all("a", class_="english-title")
        poses = [re.sub(r"[\t\n0-9.]", "", i.text.strip()) for i in find_poses]
        class_type = "Restorative"
    except AttributeError:
        info = {"Title": np.nan}
    else: 
        info = {"Title": title, "Poses": poses, "Class Type": class_type}
    return info

In [6]:
restorative_classes = [get_poses(url) for url in restorative_urls]

In [7]:
df = pd.DataFrame(restorative_classes)
df

,Title,Poses,Class Type
0,←Summer Restorative,"[Seated Corpse Pose Bolster On Blocks, Supine ...",Restorative
1,←Restorative yoga for mental clarity,"[Easy Pose, Meditation, Seated Neck Rolls, Cor...",Restorative
2,←Restorative - minutes,"[Child Pose Bolster, Supported Frog Pose Chest...",Restorative
3,←Letting Go of Expectations,"[Revolved Child Pose Variation Elbow Up, Revol...",Restorative
4,←Stretch,"[Corpse Pose, Constructive Rest Pose On Bolste...",Restorative
...,...,...,...
2523,←Releasing the senses - Pratyahara,"[Reclined Yoga Sequence, Reclined Yoga Sequenc...",Restorative
2524,←Yoga restaurativo a semana posturas de apertu...,"[Hands Chest Chair, Hand Clenches Chair, Hands...",Restorative
2525,←Restorative - Props,"[One Legged Bow Pose Preparation With Strap, S...",Restorative
2526,←Neck and Upper body Restorative Sequence,"[Bridge Pose On Blanket Block, Thunderbolt Pos...",Restorative


In [8]:
f = open("restorative_df", "wb")
pickle.dump(df, f)
f.close()